<a href="https://colab.research.google.com/github/mtsilimos/ETH/blob/main/CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
import numpy as np
from skimage.io import imread
from skimage.io import imread_collection
dirName = '/content/drive/My Drive/images/*.png'
collection1 = imread_collection(dirName)
collection1

In [88]:
image_dataset = np.array(collection1)

In [6]:
print("Image data shape is: ", image_dataset.shape)

Image data shape is:  (209,)


In [129]:
from skimage.io import imread
from skimage.io import imread_collection
dirName = '/content/drive/My Drive/masks/*.png'
collection2 = imread_collection(dirName)
collection2

In [130]:
mask_dataset = np.array(collection2)

In [8]:
print("Mask data shape is: ", mask_dataset.shape)

Mask data shape is:  (211,)


In [89]:
import cv2
import glob

for filename in glob.glob('/content/drive/My Drive/images/*.png'): # path to your images folder
    print(filename)
    img=cv2.imread(filename)
    rl1=cv2.resize(img, (160,160))
    cv2.imwrite(f'{filename}resized.png', rl1)

/content/drive/My Drive/images/105618339-super-purple-modern-suv-car-side-view.png
/content/drive/My Drive/images/156551_preview.png
/content/drive/My Drive/images/2009-Ford-Kuga-Show-Car-Side-1280x960.png
/content/drive/My Drive/images/1965-Shelby-Cobra-Roadster-Side-Profile.png
/content/drive/My Drive/images/2011-278656-pagani-huayra-side-view1.png
/content/drive/My Drive/images/2018-bm4-gt4-sports-car-side-view-pk.png
/content/drive/My Drive/images/2008-Morgan-Life-Car-Side-Speed-1024x768.png
/content/drive/My Drive/images/105618639-modern-pale-blue-suv-car-side-view.png
/content/drive/My Drive/images/178583_Side_Profile_Web.png
/content/drive/My Drive/images/003.png
/content/drive/My Drive/images/2015_aston_martin_db10_4.png
/content/drive/My Drive/images/G40-Junior-GT5-car-side.png
/content/drive/My Drive/images/2018-nissan-sentra-2017-nissan-sentra-car-nissan-micra-png-f.png
/content/drive/My Drive/images/2020-Honda-City-7-1068x672.png
/content/drive/My Drive/images/2019-acura-rd

In [90]:
import cv2
import glob

for filename in glob.glob('/content/drive/My Drive/masks/*.png'): # path to your images folder
    print(filename)
    img=cv2.imread(filename)
    rl2=cv2.resize(img, (160,160))
    cv2.imwrite(f'{filename}resized.png', rl2)

/content/drive/My Drive/masks/1965-Shelby-Cobra-Roadster-Side-Profile.png
/content/drive/My Drive/masks/2009-Ford-Kuga-Show-Car-Side-1280x960.png
/content/drive/My Drive/masks/105618639-modern-pale-blue-suv-car-side-view.png
/content/drive/My Drive/masks/2008-Morgan-Life-Car-Side-Speed-1024x768.png
/content/drive/My Drive/masks/003.png
/content/drive/My Drive/masks/156551_preview.png
/content/drive/My Drive/masks/178583_Side_Profile_Web.png
/content/drive/My Drive/masks/105618339-super-purple-modern-suv-car-side-view.png
/content/drive/My Drive/masks/94374978-pretty-pink-modern-fast-city-car-side-view.png
/content/drive/My Drive/masks/2011-278656-pagani-huayra-side-view1.png
/content/drive/My Drive/masks/2019-acura-rdx-11.png
/content/drive/My Drive/masks/676069.png
/content/drive/My Drive/masks/865-8651925_fiat-tipo-company-car-side-view.png
/content/drive/My Drive/masks/IMG_20201123_132159.png
/content/drive/My Drive/masks/63-633585_infiniti-png-car-side-profile-png-transparent-png.p

In [91]:
rl1.shape

(160, 160, 3)

In [92]:
rl2.shape

(160, 160, 3)

In [95]:
image_dataset = rl1 /255
image_dataset.shape

(160, 160, 3)

In [132]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, rl2, test_size = 0.20, random_state = 42)


In [133]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, ReLU
from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate


def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network.
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x

In [134]:
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

In [135]:
def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [136]:
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:  #Binary
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d4)  #Change the activation based on n_classes
    print(activation)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [137]:


input_shape = (160, 160, 3)


In [138]:
model = build_unet(input_shape, n_classes=1)
model.compile(optimizer=Adam(learning_rate = 1e-3), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

sigmoid
Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_18 (InputLayer)       [(None, 160, 160, 3)]        0         []                            
                                                                                                  
 conv2d_84 (Conv2D)          (None, 160, 160, 64)         1792      ['input_18[0][0]']            
                                                                                                  
 batch_normalization_26 (Ba  (None, 160, 160, 64)         256       ['conv2d_84[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_53 (Activation)  (None, 160, 160, 64)         0         ['batch_normalizat

In [139]:

history = model.fit(X_train, y_train,
                    batch_size = 16,
                    verbose=1,
                    epochs=3,
                    validation_data=(X_test, y_test),
                    shuffle=False)

Epoch 1/3


ValueError: ignored

In [120]:
input_shape = (160, 160, 3)
model = U_Net(input_shape)
model.summary()

NameError: ignored

In [119]:
model.summary()

AttributeError: ignored

In [75]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [76]:
model.fit(X_train, y_train, batch_size=32, epochs=3, validation_data=(X_test, y_test))

Epoch 1/3


ValueError: ignored

In [59]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [60]:
results = model.fit(X_train, y_train, batch_size=32, epochs=3, validation_data=(X_test, y_test))


Epoch 1/3


ValueError: ignored

In [50]:
input_shape = (160, 160, 3)
model = U_Net(input_shape)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 160, 160, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 160, 160, 64)         1792      ['input_3[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 160, 160, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 160, 160, 64)         0         ['batch_normalization[0][0

In [77]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, ReLU
from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential

In [78]:
def convolution_operation(entered_input, filters=64):

    conv1 = Conv2D(filters, kernel_size = (3,3), padding = "same")(entered_input)
    batch_norm1 = BatchNormalization()(conv1)
    acti1 = ReLU()(batch_norm1)


    conv2 = Conv2D(filters, kernel_size = (3,3), padding = "same")(acti1)
    batch_norm2 = BatchNormalization()(conv2)
    acti2 = ReLU()(batch_norm2)

    return act2

In [79]:
def encoder(entered_input, filters=64):

    encod1 = convolution_operation(entered_input, filters)
    MaxPool1 = MaxPooling2D(strides = (2,2))(encod1)
    return encod1, MaxPool1

def decoder(entered_input, skip, filters=64):
    Upsample = Conv2DTranspose(filters, (2, 2), strides=2, padding="same")(entered_input)
    Connect_Skip = Concatenate()([Upsample, skip])
    out = convolution_operation(Connect_Skip, filters)
    return out

In [84]:
def U_Net(Image_Size):
    # Take the image size and shape
    input1 = Input(Image_Size)

    # Construct the encoder blocks
    skip1, encoder_1 = encoder(input1, 64)
    skip2, encoder_2 = encoder(encoder_1, 64*2)
    skip3, encoder_3 = encoder(encoder_2, 64*4)
    skip4, encoder_4 = encoder(encoder_3, 64*8)

    # Preparing the next block
    conv_block = convolution_operation(encoder_4, 64*16)

    # Construct the decoder blocks
    decoder_1 = decoder(conv_block, skip4, 64*8)
    decoder_2 = decoder(decoder_1, skip3, 64*4)
    decoder_3 = decoder(decoder_2, skip2, 64*2)
    decoder_4 = decoder(decoder_3, skip1, 64)

    out = Conv2D(1, 1, padding="same", activation="sigmoid")(decoder_4)

    model = Model(input1, out)
    return model